# LangChain402: An L402-aware APIChain for LangChain Agents

In this notebook, the L402 APIChain wrapper for LangChain agents. This enables LangChain agents to hit APIs that are gated behind an L402 reverse proxy. The [L402 protocol](https://docs.lightning.engineering/the-lightning-network/l402) is a protocol that finally leverages the forgotten `HTTP 402 Payment Required` error code to create a programmatic way to pay for content on on the web. The content may just be normal authentication, a blog post, or even a rich API that uses LN micro payments to implement a metered API. The L402 protocol enables AI Agents to _pay_ for API-based resources using the internet's native currency. 

We'll start by importing a class we can use to access lnd, and also the API chain wrapper: 

In [1]:
from lightning import LndNode
from L402 import L402APIChain

from langchain.llms import OpenAI

In `langchain`, API chains work by consuming a description of an API, which is then passed along to a series of LLM chains to try to figure out how to call an API to answer a prompt/request. In this example, we'll create docs for a fake API running locally, which looks very similar to the popular OpenAI API structure: 


In [13]:
API_DOCS ='''BASE URL: http://localhost:8085

API Documentation
The API endpoint /v1/models HTTP endpoint can be used to fetch the set of supported models. The response is text, the list of models separated by a comma

Request:
There are no query params for the API

Response:
A JSON object of the set of supported models, which looks similar to:
{
  "models": [
    {
      "id": "gpt9",
      "name": "GPT-9",
      "version": "1.0"
    },
    {
      "id": "gpt10",
      "name": "GPT-10",
      "version": "1.0"
    },
    {
      "id": "gptalphabeta",
      "name": "GPT-Alpha-Beta",
      "version": "1.0"
    }
  ]
}
'''

We described the URL of the API, the endpoint available, and also gave it a few-shot example of what a response looks like. 

Next, we'll initialize the API chain wrapper, and also an agent that can use the L402 API Chain traverse the API. Note that the agent doesn't need to know about L402 explicitly, instead the negotiation will happen in the background

In [6]:
url = 'http://localhost:8085/v1'

lnd_node = LndNode(
    cert_path='~/gocode/src/github.com/lightningnetwork/lnd/test_lnd2/tls.cert',
    macaroon_path='~/gocode/src/github.com/lightningnetwork/lnd/test_lnd2/data/chain/bitcoin/simnet/admin.macaroon',
    host='localhost',
    port=10018
)

llm = OpenAI(temperature=0)

chain_new = L402APIChain.from_llm_and_api_docs(
    llm, API_DOCS, lightning_node=lnd_node, verbose=True,
)

First, we'll try to hit the url above using just normal `requests` to show that we need L402 awareness:

In [11]:
import requests 
resp = requests.get(url)
print(resp)
print(resp.headers)

<Response [402]>
{'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Access-Control-Allow-Headers': 'Authorization, Grpc-Metadata-macaroon, WWW-Authenticate', 'Access-Control-Allow-Methods': 'GET, POST, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'WWW-Authenticate', 'Connection': 'keep-alive', 'Content-Type': 'text/plain; charset=utf-8', 'User-Agent': 'python-requests/2.30.0', 'Www-Authenticate': 'LSAT macaroon="AgEEbHNhdAJCAAD5dBUU9CqsQKbOM72ZikBtaQemr4Pe812w/qrHyKIoQEUkoNdaqgj/pUuJ97nPbEhHvLLIZnc79FZu00m1IXvgAAIac2VydmljZXM9Z3B0LTQtYWxsLXByb3h5OjAAAi1ncHQtNC1hbGwtcHJveHlfY2FwYWJpbGl0aWVzPWdwdDMtdHVyYm8sZ3B0LTQAAhZ2YWxpZF91bnRpbD0yMDIzLTA2LTAxAAImZ3B0LTQtYWxsLXByb3h5X3ZhbGlkX3VudGlsPTE3MTg3NjMzMTkAAAYg2LCG/XVLCOF9PPYKCweoGrzqDOKageDIUfktodXpKL8=", invoice="lnsb1u1pjfz2avpp5l96p298592kypfkwxw7enzjqd45s0f40s000xhdsl64v0j9z9pqqdq8f3f5z4qcqzzsxqyz5vqsp5wmp7xry0svqv0h5xs27rd6ut50lk0xj3yeltaa2grk9cvpqx6trq9qyyssq7n68wh6swj5kc68y8v7c2s05zy28nts3na5s2w9gv

Now we'll try it with the agent: 

In [14]:
output = chain_new.run('how many total models are supported?')
print(output)

output = chain_new.run('which models are supported?')
print(output)



> Entering new  chain...
 http://localhost:8085/v1/models
L402 error path=http://localhost:8085/v1/models, attempting to pay invoice and retry request...
Paying invoice:  lnsb1u1pjfztqzpp5cta25amjvvrka0n9q6xu263k994ulhl9en8ez3qax4ufn7dxdt9sdq8f3f5z4qcqzzsxqyz5vqsp5ph0dp4kwt02g7qg7fwut3xnwexg6d4luct34m4stx90lhc4xx3es9qyyssq88ur2k9x6c9zpl40t4tnmd920sxnl59ts8p08z3gktw9sg2hxt08dr48v4a3fmufplavexp43uzgzfsmaelxkzuxgq5tu7jjg6elmrgqq2jugh
Obtained preimage:  51ec6cd0c35651ad86af89f7d839a008f7bd751f346cc2db5786d69b4d5601d3
Assembling final authorization header:  {'Authorization': 'LSAT AgEEbHNhdAJCAADC+qp3cmMHbr5lBo3FajYpa8/f5czPkUQdNXiZ+aZqy6XVq8NJXWavB1wPMhkAysyqO/SKTtsFQK/LDrSlf0fsAAIac2VydmljZXM9Z3B0LTQtYWxsLXByb3h5OjAAAi1ncHQtNC1hbGwtcHJveHlfY2FwYWJpbGl0aWVzPWdwdDMtdHVyYm8sZ3B0LTQAAhZ2YWxpZF91bnRpbD0yMDIzLTA2LTAxAAImZ3B0LTQtYWxsLXByb3h5X3ZhbGlkX3VudGlsPTE3MTg3NjMzMTkAAAYgk38yUftD8+DHAAgT966aT8wQ3jTbVgWeBgivn3STRP0=:51ec6cd0c35651ad86af89f7d839a008f7bd751f346cc2db5786d69b4d5601d3'}

  "mo